In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
from PIL import Image, ImageOps

#### Parameters

In [23]:
STILL_ALIVE_REWARD = 1
DEAD_REWARD = -1000

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 180
RESIZE_HEIGHT = 137

SHOTS_FOLDER = 'data/shots/'

#### Browser functions

In [24]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard():        
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

#     try:
    board.click()
#     except WebDriverException:
#         print('Exception')
#         return
    
    
def getScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def makeMove(m):
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    if m == 1:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == 2:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == 3:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == 4:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

def makeRandomMove():
    r = random.randint(0, 4)
    makeMove(r)
    
    return r

#### Preprocess functions

In [25]:
def getLastGameIndex():
    '''Get the index of the last current game.'''
    files = os.listdir(SHOTS_FOLDER)
    last = files[-1]
    
    return parseName(last)[0]


def getName(g, i, m, r):
    '''Form the file name from the board data.'''
    return str(g).zfill(3) + '_' + str(i).zfill(3) + '_' + str(m) + '_' + str(r) + '.png'


def parseName(fileName):
    '''Parse the board data from the file name.'''
    return [int(d) for d in fileName[:-4].split('_')]


def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)
        

def preprocImg(fileName):
    '''Preprocess a screenshot.'''
    im = Image.open(fileName)    
    
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = preprocImg(fileName)
            im.save(fileName)

In [45]:
preprocAll(0)

#### Play game

In [33]:
url = 'https://playsnake.org/'

browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
# browser = webdriver.Chrome(executable_path='D:/Biblioteci/Python/chromedriver_win32/chromedriver.exe')
browser.get(url) 

START_GAME = getLastGameIndex() + 1
gameIndex = START_GAME

maxGames = 2

for g in range(maxGames):
    
    # Select the level
    selectLevel(1)
    time.sleep(2)
    
    score = 0
    screenIndex = 0
    
    clickBoard()
    while getState() == 'playing' or getState() == 'paused':
        s2 = getScreen('prev.png')
        screenIndex += 1    

        clickBoard()
        m = makeRandomMove()
        clickBoard()
        
        # Update last board  
        r = max(STILL_ALIVE_REWARD, getScore() - score) 
        fileName = getName(gameIndex, screenIndex, m, r)
        os.rename(SHOTS_FOLDER + 'prev.png', SHOTS_FOLDER + fileName)
        score = getScore()
        
    # Change reward of last board
    g, i, m, r = parseName(fileName)
    os.rename(SHOTS_FOLDER + fileName, SHOTS_FOLDER + getName(g, i, m, DEAD_REWARD))
    
    time.sleep(1)
    
    gameIndex += 1

browser.quit()

preprocAll(START_GAME)

#### Q model

In [44]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.backend import set_image_data_format
import numpy as np

set_image_data_format('channels_first')

Intai fac un model care doar prezice daca un board reprezinta un joc care s-a terminat (lovit de zid) sau nu

In [116]:
def readData():
    files = os.listdir(SHOTS_FOLDER)
    
    labels = np.zeros(len(files))    
    data = []
    
    for i, f in enumerate(files):
        im = Image.open(SHOTS_FOLDER + f)
        
        if f in history:
            labels[i] = history[f][2]
        else:
            labels[i] = STILL_ALIVE_REWARD
            
        data.append([np.asarray(im.convert("L"))])
        
    data = np.asarray(data)
    return data, labels


def sampleData(nSamples):
    files = os.listdir(SHOTS_FOLDER)
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseName(f)[3]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    PERCD = 8
    PERCA = 0.5
    PERCR = 0.5
    n = PERCD * nd + PERCA * na + PERCR * nr
    pd = PERCD / n
    pa = PERCA / n
    pr = PERCR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    return samples

def readSamples(samples):
    data = []
    labels = []
    
    for file in samples:
        im = Image.open(SHOTS_FOLDER + file)
        g, i, m, r = parseName(file)
        
        data.append([np.asarray(im.convert("L"))])
        labels.append(1 if r > 0 else -1)
        
    data = np.asarray(data)
    return data, labels
    
        
def nn_model():
    model = Sequential()
    
    model.add(Conv2D(20, (5, 5), input_shape=(1, RESIZE_HEIGHT, RESIZE_WIDTH), activation='relu', name='conv1'))
    model.add(MaxPool2D(pool_size=(4,4), name='pool1'))
    
    model.add(Conv2D(10, (5, 5), activation='relu', name='conv2'))
    model.add(MaxPool2D(pool_size=(4,4), name='pool2'))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dense(1, activation='softmax', name='dense2'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer = 'adam',
                 metrics=['binary_accuracy'])
    
    return model


In [86]:
s = sampleData(50)
readSamples(s)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1]


([[array([[  0,   0,   0, ...,   0,   0,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          ...,
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)],
  [array([[  0,   0,   0, ...,   0,   0,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          ...,
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)],
  [array([[  0,   0,   0, ...,   0,   0,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          ...,
          [  0, 255, 255, ..., 255, 255,   0],
          [  0, 255, 255, ..., 255, 255,   0],
          [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)],
  [array([[  0,   0,   0, ...,   0,   0,   0],
          [  0

In [120]:
model = nn_model()

data, labels = readSamples(sampleData(200))
print(data.shape)
print(labels)

model.fit(data, labels, epochs=5, batch_size=8, shuffle=True)

(200, 1, 137, 180)
[1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]
Epoch 1/5
200/200 [==============================] - 13s 64ms/step - loss: 9.0872 - binary_accuracy: 0.7150
Epoch 2/5
200/200 [==============================] - 12s 60ms/step - loss: 9.0872 - binary_accuracy: 0.7150
Epoch 3/5
200/200 [==============================] - 12s 60ms/step - loss: 9.0872 - binary_accuracy: 0.7150

In [118]:
model.evaluate(data[30:50], labels[30:50])

20/20 [==============================] - 1s 43ms/step


[14.348146438598633, 0.550000011920929]

In [121]:
print(model.predict(data[30:50]))
print(labels[30:50])

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1]
